# Building a Local Events Finder with Hyperbrowser and GPT-4o

In this cookbook, we'll create an intelligent events finder that can automatically discover and organize local events in any city around the world. This agent will:

1. Take a city name as input
2. Intelligently determine the country based on the city name
3. Extract event information from event platforms like Eventbrite
4. Present a curated list of events in a user-friendly format
5. Filter events based on user interests and preferences

This approach combines:
- **[Hyperbrowser](https://hyperbrowser.ai)** for web scraping and data extraction
- **OpenAI's GPT-4o** for intelligent analysis and content curation

By the end of this cookbook, you'll have a versatile tool that can help you or your users discover interesting events in any location!

## Prerequisites

Before starting, you'll need:

1. A Hyperbrowser API key (sign up at [hyperbrowser.ai](https://hyperbrowser.ai) if you don't have one)
2. An OpenAI API key with access to GPT-4o

Store these API keys in a `.env` file in the same directory as this notebook:

```
HYPERBROWSER_API_KEY=your_hyperbrowser_key_here
OPENAI_API_KEY=your_openai_key_here
```

## Step 1: Set up imports and load environment variables

We start by importing the necessary packages and initializing our environment variables. The key libraries we'll use include:
- `asyncio` for handling asynchronous operations
- `hyperbrowser` for web scraping and event extraction
- `openai` for content analysis and organization
- `pydantic` for data validation and structured responses

In [30]:
import asyncio
import json
import os

from dotenv import load_dotenv
from hyperbrowser import AsyncHyperbrowser
from hyperbrowser.models.scrape import StartScrapeJobParams, ScrapeOptions
from hyperbrowser.models.session import CreateSessionParams
from openai import AsyncOpenAI
from openai.types.chat import (
    ChatCompletionMessageParam,
    ChatCompletionMessageToolCall,
    ChatCompletionToolMessageParam,
    ChatCompletionToolParam,
)
from pydantic import BaseModel
from typing import Optional, List

load_dotenv()

True

## Step 2: Initialize API clients

Here we create instances of the Hyperbrowser and OpenAI clients using our API keys. These clients will be responsible for web scraping and intelligent analysis respectively.

In [31]:
hb = AsyncHyperbrowser(api_key=os.getenv("HYPERBROWSER_API_KEY"))
llm = AsyncOpenAI()

## Step 3: Define data models for event extraction

We'll define Pydantic models to structure our event data. This ensures that we receive consistent, well-formatted information about each event, including:
- Event title and description
- Location details
- Date and time information
- URL for more details
- Optional information like image URLs and event status

In [32]:
class EventExtractSchema(BaseModel):
    title: str
    description: str
    location: str
    date: str
    time: str
    url: str
    image_url: Optional[str]
    status: Optional[str]


class EventExtractListSchema(BaseModel):
    events: List[EventExtractSchema]


EXTRACT_EVENT_NAME = "extract_events"

EventExtractionTool: ChatCompletionToolParam = {
    "type": "function",
    "function": {
        "name": EXTRACT_EVENT_NAME,
        "description": "Extract events from a webpage",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {"type": "string"},
                "country": {"type": "string"},
            },
            "required": ["city", "country"],
        },
    },
}

MAX_INPUT_CHARACTERS = 10000

## Step 4: Create the event extraction function

This function handles the core functionality of our events finder:

1. It takes a city and country as input
2. Formats them to work with event listing sites like Eventbrite
3. Uses Hyperbrowser to scrape the event listings page
4. Employs GPT-4o mini to parse the raw HTML/markdown into structured event data
5. Returns a list of events in a consistent format

The function handles pagination limits intelligently by truncating extremely long scraped content.

In [33]:
async def handle_events_extraction(
    city: str, country: str
) -> None | EventExtractListSchema:
    country_formatted = country.replace(" ", "-").lower()
    city_formatted = city.replace(" ", "-").lower()

    url = f"https://www.eventbrite.com/d/{country_formatted}--{city_formatted}/all-events/"
    print(url)
    response = await hb.scrape.start_and_wait(
        StartScrapeJobParams(
            url=url,
            scrape_options=ScrapeOptions(formats=["markdown"]),
            session_options=CreateSessionParams(use_proxy=True),
        )
    )
    print(response)
    if response.data is None or response.data.markdown is None:
        return None
    else:
        messages: List[ChatCompletionMessageParam] = [
            {
                "role": "system",
                "content": "You are a text parser. You will be give markdown text, and you will need to parse it into a structured format.",
            },
            {
                "role": "user",
                "content": (
                    response.data.markdown
                    if len(response.data.markdown) < MAX_INPUT_CHARACTERS
                    else response.data.markdown[:MAX_INPUT_CHARACTERS]
                ),
            },
        ]
        response = await llm.beta.chat.completions.parse(
            model="gpt-4o-mini",
            messages=messages,
            response_format=EventExtractListSchema,
        )
        return response.choices[0].message.parsed

## Step 5: Implement the tool handler

The tool handler function processes requests from the LLM to interact with our event extraction functionality. It:

1. Receives tool call parameters from the LLM
2. Validates the input parameters (city and country)
3. Executes the event extraction function
4. Formats and returns the results
5. Handles any errors that might occur during execution

In [34]:
async def handle_tool_call(
    tc: ChatCompletionMessageToolCall,
) -> ChatCompletionToolMessageParam:
    print(f"Handling tool call: {tc.function.name}")

    try:
        if tc.function.name == EXTRACT_EVENT_NAME:
            args = json.loads(tc.function.arguments)
            content = await handle_events_extraction(args["city"], args["country"])
            if content is None:
                return {
                    "role": "tool",
                    "tool_call_id": tc.id,
                    "content": "No events found",
                }
            else:
                return {
                    "role": "tool",
                    "tool_call_id": tc.id,
                    "content": content.model_dump_json(),
                }
        else:
            raise ValueError(f"Tool not found: {tc.function.name}")

    except Exception as e:
        err_msg = f"Error handling tool call: {e}"
        print(err_msg)
        return {
            "role": "tool",
            "tool_call_id": tc.id,
            "content": err_msg,
            "is_error": True,  # type: ignore
        }

## Step 6: Create the agent loop

Now we implement the core agent loop that orchestrates the conversation between:
1. The user (who asks about events)
2. The LLM (which analyzes the request and determines what information is needed)
3. Our tool (which fetches and structures the event data)

This recursive pattern allows for sophisticated interactions where the agent can refine its understanding of user preferences.

In [35]:
async def agent_loop(messages: list[ChatCompletionMessageParam]) -> str:
    while True:
        response = await llm.chat.completions.create(
            messages=messages,
            model="gpt-4o",
            tools=[EventExtractionTool],
            max_completion_tokens=8000,
        )

        choice = response.choices[0]

        # Append response to messages
        messages.append(choice.message)  # type: ignore

        # Handle tool calls
        if (
            choice.finish_reason == "tool_calls"
            and choice.message.tool_calls is not None
        ):
            tool_result_messages = await asyncio.gather(
                *[handle_tool_call(tc) for tc in choice.message.tool_calls]
            )
            messages.extend(tool_result_messages)

        elif choice.finish_reason == "stop" and choice.message.content is not None:
            return choice.message.content

        else:
            print(choice)
            raise ValueError(f"Unhandled finish reason: {choice.finish_reason}")

## Step 7: Design the system prompt

The system prompt is crucial for guiding the LLM's behavior. Our prompt establishes the agent as an event finder that can:
1. Infer the country from a city name
2. Use the event extraction tool to gather event information
3. Filter events based on user preferences
4. Format the results in a user-friendly way

This provides a consistent framework for all agent interactions.

In [36]:
SYSTEM_PROMPT = """
You are an event finder. You have access to a 'extract_events' tool which can be used to get structured data from a webpage regarding events in a city. To use this, you will be given a city name - {city}. You will then infer the country from the city name and use the 'extract_events' tool to get the events in the city.

Once you have the information, you will then format the information in a way that is easy to understand and use, ideally in a markdown format.

The user may or may not provide you with a filter. If they do, you will filter the events based what you can infer from the user prompt. 

In summary, you will:
1. Infer the country from the city name
2. Use the 'extract_events' tool to get the events in the city
3. If the user provides a filter, you will filter the events based on the user prompt
   - Filter the events based on the user prompt
4. Return the information in a way that is easy to understand and use, ideally in a markdown format
""".strip()

## Step 8: Create a factory function for generating event finders

Now we'll create a factory function that generates a specialized event finder for any city. This function:

1. Takes a city name as input
2. Formats the system prompt with this value
3. Returns a function that can answer questions about events in that location

This approach makes our solution reusable for event discovery in any location worldwide.

In [37]:
from typing import Coroutine, Any, Callable


def make_event_finder(city: str) -> Callable[..., Coroutine[Any, Any, str]]:
    sysprompt = SYSTEM_PROMPT.format(
        city=city,
    )

    async def event_finder(question: str) -> str:
        messages: list[ChatCompletionMessageParam] = [
            {"role": "system", "content": sysprompt},
        ]

        if question:
            messages.append({"role": "user", "content": question})

        return await agent_loop(messages)

    return event_finder

## Step 9: Test the events finder

Let's test our agent by searching for events in San Francisco. This demonstrates the full workflow:

1. The agent infers that San Francisco is in the United States
2. It searches for events in San Francisco
3. It extracts and formats the event information
4. It presents a curated list of events

For this example, we're looking specifically for gaming events in San Francisco.

In [38]:
city = "San Francisco"
event_finder = make_event_finder(city)
response = await event_finder("I'm looking for gaming events in San Francisco")

Handling tool call: extract_events


## Step 10: Display the results

Finally, we'll display the formatted results from our agent. The response is already in markdown format, making it easy to read and navigate. This allows users to quickly scan the list of events and find those that interest them most.

In [41]:
from IPython.display import Markdown, display

if response is not None:
    display(Markdown(response))
else:
    print("No events found")

Here are some upcoming gaming events in San Francisco:

### Courage XL 2025 – Indie Showcase & GDC Pre-Party
- **Description:** Join us for an indie showcase ahead of GDC 2025.
- **Location:** The Great Northern
- **Date & Time:** Sunday, March 16, 7:00 PM
- **[Event Link](https://www.eventbrite.nl/e/courage-xl-2025-indie-showcase-gdc-pre-party-tickets-1152458246639)**
- ![Image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F945517783%2F3673532979%2F1%2Foriginal.20250127-142718?w=512&auto=format%2Ccompress&q=75&sharp=10&rect=4%2C59%2C1916%2C958&s=49729210ab1a5a8e42a2426b04b57a68)

### MeetToMatch - The San Francisco Edition 2025, powered by Xsolla
- **Description:** A networking event for professionals in the gaming industry.
- **Location:** Moscone Center
- **Date & Time:** Monday, March 17, 9:00 AM
- **[Event Link](https://www.eventbrite.nl/e/meettomatch-the-san-francisco-edition-2025-powered-by-xsolla-tickets-1043787264807)**
- ![Image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F894705613%2F246618256503%2F1%2Foriginal.20241108-113241?crop=focalpoint&fit=crop&w=512&auto=format%2Ccompress&q=75&sharp=10&fp-x=0.005&fp-y=0.005&s=ec76d80a6c2a035246d7a7167a12f809)

### Indie BYOG Showcase @ GDC2025 Presented by Onerat Games & Sub-Zero Sound
- **Description:** A showcase for indie games during GDC 2025.
- **Location:** BuzzWorks
- **Date & Time:** Tuesday, March 18, 6:00 PM
- **[Event Link](https://www.eventbrite.com.au/e/indie-byog-showcase-gdc2025-presented-by-onerat-games-sub-zero-sound-tickets-1220801142079)**
- ![Image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F960591873%2F343842521485%2F1%2Foriginal.20250214-041703?w=512&auto=format%2Ccompress&q=75&sharp=10&rect=0%2C0%2C940%2C470&s=5857017a2587c787bd57b3c58c7e6b89)

### Pocket Gamer Connects San Francisco 2025
- **Description:** A major event for mobile gaming industry professionals.
- **Location:** The Hibernia
- **Date & Time:** Monday, March 17, 9:00 AM
- **[Event Link](https://www.eventbrite.co.uk/e/pocket-gamer-connects-san-francisco-2025-tickets-1118784036169)**
- ![Image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F920901043%2F2120415922323%2F1%2Foriginal.20241220-115451?crop=focalpoint&fit=crop&w=512&auto=format%2Ccompress&q=75&sharp=10&fp-x=0.545454545455&fp-y=0.623605947955&s=2bbf93062278f409f8dca50008545715)

### Australian Showcase @ GDC 2025 Presented by Indie BYOG & IGEA
- **Description:** A showcase featuring Australian game developers during GDC 2025.
- **Location:** BuzzWorks
- **Date & Time:** Monday, March 17, 6:00 PM
- **[Event Link](https://www.eventbrite.com.au/e/australian-showcase-gdc-2025-presented-by-indie-byog-igea-tickets-1222569721949)**
- ![Image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F946442813%2F188659113760%2F1%2Foriginal.20250128-113310?crop=focalpoint&fit=crop&w=512&auto=format%2Ccompress&q=75&sharp=10&fp-x=0.5&fp-y=0.5&s=ed0ef77903accc8e452d5730d0fcb425)

These are some of the prominent gaming events happening soon in San Francisco. Let me know if you need more details or assistance!

## Conclusion

In this cookbook, we built a powerful events finder using Hyperbrowser and GPT-4o. This agent can:

1. Automatically search for events in any city worldwide
2. Extract and structure information about each event
3. Filter events based on user interests
4. Present results in a clean, readable format

This pattern can be extended to create more sophisticated event discovery tools, such as:
- Event finders for specific types of events (concerts, workshops, etc.)
- Personalized event recommendations based on user preferences
- Multi-platform event aggregation from various sources

### Next Steps

To take this further, you might consider:
- Adding support for date-based filtering
- Implementing price range filtering
- Creating a web interface for easier interaction
- Extending to multiple event platforms beyond Eventbrite
- Adding calendar integration for saving interesting events

Happy event hunting! 🎉

## Relevant Links
- [Hyperbrowser](https://hyperbrowser.ai)
- [OpenAI Docs](https://platform.openai.com/docs/introduction)